In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
 
import random
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import cv2
import PIL
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tqdm import tqdm

In [2]:
# load crop data pairs

# Paths to your data.
pos_path = './archive/cropped/CFD_background_region/'
neg_path = './archive/cropped/CFD_background_region/'
img_size = (112, 112)

# Load and preprocess data
def load_data_pairs(pos_path, neg_path):
    pos_images= []  # List to store the images
    neg_images = []   # List to store the masks
    pairList = []
    files = []
    count = 0
    for filename in os.listdir(pos_path):
        if count > 20:
            break
        pos_img_path = pos_path + filename
        neg_img_path = neg_path + filename  # Assuming mask has same filename
        # print(img_path)
        # Load and preprocess the image
        pos_img = cv2.imread(pos_img_path)
        neg_img = cv2.imread(neg_img_path)
        # print(img.size())
    #   if not img or not mask:
    #         continue
        pos_img, neg_img = pos_img/255.0,  neg_img/255.0
        # img = cv2.resize(img, img_size) / 255.0
        # # Load and preprocess the mask using OpenCV
        # mask = cv2.resize(mask, img_size) / 255.0
 
        # for i in range(4):
        #     for j in range(4):
        #         img_part = pos_img[i * 112: (i + 1) * 112,
        #                     j * 112: (j + 1) * 112]
        #         mask_part = neg_img[i * 112: (i + 1) * 112,
        #                     j * 112: (j + 1) * 112]
        pos_images.append(pos_img)
        neg_images.append(neg_img)
        files.append(filename)
         
        count += 1
 
    return np.array(pos_images), np.array(neg_images)
 

pos_img, neg_img = load_data_pairs(pos_path, neg_path)
pos_ds = tf.data.Dataset.from_tensor_slices(pos_img)
neg_ds = tf.data.Dataset.from_tensor_slices(neg_img)

# Zip the positive and negative datasets together to create pairs
train_dataset = tf.data.Dataset.zip((pos_ds, neg_ds))

# Shuffle and batch the dataset as needed
buffer_size = len(pos_img)  # This should be the number of images, can be replaced with a suitable buffer size
batch_size = 8  # Define your batch size as needed
train_dataset = train_dataset.shuffle(buffer_size=buffer_size).batch(batch_size)

# Prefetch data for performance optimization
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
 

improved unet

In [24]:
# Residual block
# def residual_block(input_tensor, num_filters):
#     x = layers.Conv2D(num_filters, 3, padding="same")(input_tensor)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)

#     x = layers.Conv2D(num_filters, 3, padding="same")(x)
#     x = layers.BatchNormalization()(x)

#     # Add the input_tensor (residual connection) to the output of the convolution block
#     x = layers.add([x, input_tensor])
#     x = layers.ReLU()(x)

#     return x
def residual_block(input_tensor, num_filters):
    # Check if a projection shortcut is needed
    input_shape = input_tensor.shape[-1]
    if input_shape != num_filters:
        shortcut = layers.Conv2D(num_filters, (1, 1), strides=(1, 1), padding='same')(input_tensor)
    else:
        shortcut = input_tensor

    x = layers.Conv2D(num_filters, 3, padding="same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Add the shortcut (which has been adjusted if necessary) to the output of the convolution block
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)

    return x
# Attention gate
def attention_gate(input_tensor, gate_tensor, num_filters):
    # Resize gate tensor to match the input_tensor shape using a 1x1 convolution
    gate_resized = layers.Conv2D(num_filters, 1, padding='same')(gate_tensor)
    gate_resized = layers.BatchNormalization()(gate_resized)
    gate_resized = layers.ReLU()(gate_resized)

    # Add the gate to the input_tensor
    x = layers.add([input_tensor, gate_resized])
    attention = layers.Conv2D(num_filters, 1, padding='same', activation='sigmoid')(x)
    x = layers.multiply([input_tensor, attention])

    return x

def conv_block(input_tensor, num_filters):
    x = residual_block(input_tensor, num_filters)
    return x

def encoder_block(input_tensor, num_filters):
    x = conv_block(input_tensor, num_filters)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

# Modify the decoder_block to include an attention gate
def decoder_block(input_tensor, skip_features, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input_tensor)

    # Apply attention gate to the skip features before concatenating
    attention_skipped = attention_gate(skip_features, x, num_filters)

    # Concatenate upsampled input with the attention modified skip features
    x = layers.concatenate([x, attention_skipped])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = layers.Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = conv_block(p4, 1024)

    # Decoder
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Output
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = models.Model(inputs, outputs)
    return model

input_shape = (112, 112, 3)
model = build_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


encoder

In [4]:
def build_encoder(input_shape):
    inputs = layers.Input(shape=input_shape)
    x, _ = encoder_block(inputs, 64)
    x, _ = encoder_block(x, 128)
    x, _ = encoder_block(x, 256)
    x, _ = encoder_block(x, 512)
    x = conv_block(x, 1024)
    x = layers.GlobalAveragePooling2D()(x)
    return keras.Model(inputs, x, name="encoder")

# Define the projection head using the functional API
def build_projection_head(encoder):
    inputs = layers.Input(shape=(1024,))
    x = layers.Dense(512, activation="relu")(inputs)
    outputs = layers.Dense(256)(x)
    return keras.Model(inputs, outputs, name="projection_head")

# Get the models
input_shape = (112, 112, 3)
encoder_model = build_encoder(input_shape)
projection_head_model = build_projection_head(encoder_model)

contrastive_loss

In [5]:
import tensorflow as tf

# Cosine similarity function
def cosine_similarity(z1, z2):
    z1 = tf.math.l2_normalize(z1, axis=1)
    z2 = tf.math.l2_normalize(z2, axis=1)
    return tf.matmul(z1, z2, transpose_b=True)
def contrastive_loss(z1, z2, temperature=0.5):
    batch_size = tf.shape(z1)[0]

    # Compute the cosine similarity matrix between z1 and z2
    cosine_sim = cosine_similarity(z1, z2)

    # Compute the cosine similarity within each batch for negative pairs
    cosine_sim_z1_z1 = cosine_similarity(z1, z1)
    cosine_sim_z2_z2 = cosine_similarity(z2, z2)

    # Mask to clear out the diagonal and keep only the negative pairs
    mask = 1 - tf.eye(batch_size)
    neg_sim_z1 = mask * cosine_sim_z1_z1
    neg_sim_z2 = mask * cosine_sim_z2_z2

    # Summing negatives from both transformations for each sample
    sum_neg_sim_z1 = tf.reduce_sum(tf.exp(neg_sim_z1 / temperature), axis=1)
    sum_neg_sim_z2 = tf.reduce_sum(tf.exp(neg_sim_z2 / temperature), axis=1)

    # Positive pairs (diagonal of the cosine similarity matrix between z1 and z2)
    pos_cosine_sim = tf.linalg.diag_part(cosine_sim)
    exp_positives = tf.exp(pos_cosine_sim / temperature)

    # Contrastive loss for each pair （L1 +L2)
    loss_per_sample = -tf.math.log(exp_positives / sum_neg_sim_z1) - tf.math.log(exp_positives / sum_neg_sim_z2)

    # Average loss over all pairs
    loss = tf.reduce_mean(loss_per_sample)

    return loss
# # Contrastive loss function
# def contrastive_loss(z1, z2, temperature=0.5):
#     # Get the number of samples in the batch
#     batch_size = tf.shape(z1)[0]

#     # Calculate cosine similarity within the batch
#     cosine_sim = cosine_similarity(z1, z2)

#     # Positive samples similarity: diagonal elements of cosine similarity matrix
#     pos_sim = tf.linalg.diag_part(cosine_sim)

#     # Negative samples similarity: off-diagonal elements of cosine similarity matrix
#     # By setting the diagonal to a large negative value, we effectively ignore the positive sample
#     # similarities when calculating the denominator of the contrastive loss
#     mask_off_diag = 1 - tf.eye(batch_size)
#     neg_sim = cosine_sim * mask_off_diag

#     # Exponential of the positive samples similarity
#     exp_pos_sim = tf.exp(pos_sim / temperature)

#     # Sum of exponentials of the negative samples similarity
#     exp_neg_sim_sum = tf.reduce_sum(tf.exp(neg_sim / temperature), axis=1)

#     # Loss calculation for each sample
#     loss_per_sample = -tf.math.log(exp_pos_sim / (exp_pos_sim + exp_neg_sim_sum))

#     # Mean loss over all samples in the batch
#     loss = tf.reduce_mean(loss_per_sample)

#     return loss

# Encoder and projection head should be defined here (omitted for brevity)

# Input Tensors
inputs = tf.keras.Input(shape=(112, 112, 3))

# Forward pass through the encoder and projection head
encoder = build_encoder(input_shape=(112, 112, 3))
encoder_output_shape = encoder.output_shape[1]
projection_head = build_projection_head(encoder_output_shape)

encoded = encoder(inputs)
projected = projection_head(encoded)

# Define the Keras model
model = tf.keras.Model(inputs=inputs, outputs=projected)

# Compile the model with a dummy optimizer and loss since we are using a custom training loop
model.compile(optimizer='adam', loss=None)

num_epochs = 10
# Custom training loop
optimizer = tf.keras.optimizers.Adam()
for epoch in range(num_epochs):
    
    print(f"Epoch {epoch + 1}/{num_epochs}")
    pbar = tqdm(total=len(train_dataset), desc='Training', leave=True)
    
    for step, (images_pos, images_neg) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            proj_pos = model(images_pos, training=True)  # Projection of positive samples
            proj_neg = model(images_neg, training=True)  # Projection of negative samples
            loss_value = contrastive_loss(proj_pos, proj_neg, temperature=0.5)

        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        pbar.update(1)
        pbar.set_postfix(loss=f"{loss_value.numpy():.4f}") 
    pbar.close()
        

Epoch 1/10


Training: 100%|██████████| 3/3 [03:30<00:00, 70.12s/it, loss=-0.3846]


Epoch 2/10


Training: 100%|██████████| 3/3 [03:11<00:00, 63.73s/it, loss=-0.4315]


Epoch 3/10


...

Training: 100%|██████████| 3/3 [03:15<00:00, 65.09s/it, loss=-0.6406]


Epoch 9/10


Training: 100%|██████████| 3/3 [03:12<00:00, 64.27s/it, loss=-0.8439]


Epoch 10/10


Training: 100%|██████████| 3/3 [03:10<00:00, 63.45s/it, loss=-0.6914]


In [26]:
encoder_layers = [layer for layer in encoder_model.layers if 'conv' in layer.name or 'pooling' in layer.name]
model_encoder_layers = [layer for layer in model.layers if 'conv' in layer.name or 'pooling' in layer.name][:len(encoder_layers)]
for unet_layer, encoder_layer in zip(model_encoder_layers, encoder_layers):
    unet_layer.set_weights(encoder_layer.get_weights())

ValueError: You called `set_weights(weights)` on layer 'max_pooling2d_12' with a weight list of length 2, but the layer was expecting 0 weights.

In [7]:

# Paths to your data.
images_path = './archive/crack_segmentation_dataset/train/images/'
masks_path = './archive/crack_segmentation_dataset/train/masks/'
img_size = (112, 112)

# Load and preprocess data
def load_data(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    files = []
    count = 0
    for filename in os.listdir(images_path):
        if count > 500:
            break
        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename
        # print(img_path)
        # Load and preprocess the image
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        # print(img.size())
    #   if not img or not mask:
    #         continue
        img, mask = img/255.0, mask/255.0
        # img = cv2.resize(img, img_size) / 255.0
        # # Load and preprocess the mask using OpenCV
        # mask = cv2.resize(mask, img_size) / 255.0
 
        for i in range(4):
            for j in range(4):
                img_part = img[i * 112: (i + 1) * 112,
                            j * 112: (j + 1) * 112]
                mask_part = mask[i * 112: (i + 1) * 112,
                            j * 112: (j + 1) * 112]
                images.append(img_part)
                masks.append(mask_part)
        files.append(filename)
         
        count += 1
    print(files)
    return np.array(images), np.array(masks)


In [8]:
def load_test(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    count = 0
    print()
    imgList = os.listdir(images_path)
    files = []
    for i in range(len(imgList) - 1, -1, -1):
        filename = imgList[i]
    # for filename in os.listdir(images_path):
        if count > 100:
            break

        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename

        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        img, mask = img/255.0, mask/255.0

        i, j = random.randint(0, 3), random.randint(0, 3)
        img_part = img[i * 112: (i + 1) * 112,
                    j * 112: (j + 1) * 112]
        mask_part = mask[i * 112: (i + 1) * 112,
                    j * 112: (j + 1) * 112]
        images.append(img_part)
        masks.append(mask_part)

        files.append(filename)
        count += 1
    print(files)
    return np.array(images), np.array(masks)

In [14]:
# Load your data
images, masks = load_data(images_path, masks_path, (112, 112))


['CFD_002.jpg', 'CFD_003.jpg', 'CFD_004.jpg', 'CFD_005.jpg', 'CFD_006.jpg', 'CFD_008.jpg', 'CFD_009.jpg', 'CFD_010.jpg', 'CFD_012.jpg', 'CFD_015.jpg', 'CFD_016.jpg', 'CFD_017.jpg', 'CFD_018.jpg', 'CFD_020.jpg', 'CFD_021.jpg', 'CFD_022.jpg', 'CFD_023.jpg', 'CFD_024.jpg', 'CFD_025.jpg', 'CFD_026.jpg', 'CFD_027.jpg', 'CFD_028.jpg', 'CFD_029.jpg', 'CFD_030.jpg', 'CFD_031.jpg', 'CFD_032.jpg', 'CFD_033.jpg', 'CFD_034.jpg', 'CFD_035.jpg', 'CFD_036.jpg', 'CFD_038.jpg', 'CFD_039.jpg', 'CFD_041.jpg', 'CFD_042.jpg', 'CFD_043.jpg', 'CFD_044.jpg', 'CFD_045.jpg', 'CFD_046.jpg', 'CFD_048.jpg', 'CFD_049.jpg', 'CFD_050.jpg', 'CFD_051.jpg', 'CFD_052.jpg', 'CFD_053.jpg', 'CFD_054.jpg', 'CFD_055.jpg', 'CFD_056.jpg', 'CFD_057.jpg', 'CFD_058.jpg', 'CFD_059.jpg', 'CFD_060.jpg', 'CFD_061.jpg', 'CFD_062.jpg', 'CFD_063.jpg', 'CFD_064.jpg', 'CFD_065.jpg', 'CFD_066.jpg', 'CFD_067.jpg', 'CFD_068.jpg', 'CFD_069.jpg', 'CFD_071.jpg', 'CFD_072.jpg', 'CFD_073.jpg', 'CFD_074.jpg', 'CFD_075.jpg', 'CFD_076.jpg', 'CFD_077.

In [15]:
images.shape

(8016, 112, 112, 3)

In [16]:
images1, masks1 = images[:500], masks[:500]
masks1 = np.expand_dims(masks1, axis=-1)

In [17]:

# Load your data
imgval, maskval = load_test(images_path, masks_path, (112, 112))

maskval = np.expand_dims(maskval, axis=-1)


['Volker_DSC01710_725_197_1392_1586.jpg', 'Volker_DSC01710_608_6_1524_1817.jpg', 'Volker_DSC01710_521_437_1123_892.jpg', 'Volker_DSC01710_449_161_1824_1502.jpg', 'Volker_DSC01710_37_518_1605_1304.jpg', 'Volker_DSC01710_340_159_2049_1635.jpg', 'Volker_DSC01710_15_28_1388_1132.jpg', 'Volker_DSC01710_112_246_1993_1548.jpg', 'Volker_DSC01710_1087_22_1502_1742.jpg', 'Volker_DSC01710_1025_76_1293_1327.jpg', 'Volker_DSC01710_0_0_2736_1824.jpg', 'Volker_DSC01709_599_1337_1214_1373.jpg', 'Volker_DSC01709_525_1036_1256_1448.jpg', 'Volker_DSC01709_43_382_1337_1425.jpg', 'Volker_DSC01709_1_428_1791_1806.jpg', 'Volker_DSC01709_191_287_1561_2056.jpg', 'Volker_DSC01709_176_298_1435_1635.jpg', 'Volker_DSC01709_14_103_1566_1955.jpg', 'Volker_DSC01709_109_399_1680_2130.jpg', 'Volker_DSC01709_0_0_1824_2736.jpg', 'Volker_DSC01708_820_247_911_1098.jpg', 'Volker_DSC01708_7_339_1805_2017.jpg', 'Volker_DSC01708_6_308_1788_2155.jpg', 'Volker_DSC01708_3_1033_1692_1349.jpg', 'Volker_DSC01708_26_342_1730_1845.jp

In [18]:
masks1.shape

(500, 112, 112, 1)

In [19]:

X_train, y_train, X_val, y_val = images1, masks1,imgval, maskval 
# y_train.shape
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (500, 112, 112, 3)
y_train shape: (500, 112, 112, 1)
X_val shape: (101, 112, 112, 3)
y_val shape: (101, 112, 112, 1)


In [20]:
images1[:200,:,:,:].shape
 

(200, 112, 112, 3)

In [21]:
import numpy as np

# 假设 images1.shape == (800, 112, 112, 3)
num_samples = 200
 
pos_pairs = images1[:num_samples,:,:,:] 
neg_pairs = images1[num_samples:2*num_samples,:,:,:] 
print(pos_pairs.shape)                  
 
assert not np.isnan(pos_pairs).any() and not np.isnan(neg_pairs).any()
  
 
dummy_y = np.zeros((num_samples, 256))
# 训练模型
pretrain_model.fit([pos_pairs, neg_pairs], dummy_y, epochs=10)

(200, 112, 112, 3)


NameError: name 'pretrain_model' is not defined

In [ ]:
pretrain_model.fit([images1[:200,:,:,:], images1[200:400,:,:,:]], epochs=10)

Epoch 1/10


ValueError: None values not supported.

In [25]:


# Create data generators with desired augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()  # No augmentation for validation data

# Generate batches of augmented data from arrays (X_train and y_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=8)
val_generator = val_datagen.flow(X_val, y_val, batch_size=8)

# Train the model using the fit_generator function
history = model.fit(
    train_generator, 
    steps_per_epoch=len(X_train) // 8,  # Number of batches per epoch
    epochs=10, 
    validation_data=val_generator,
    validation_steps=len(X_val) // 8  # Number of batches for validation
)


Epoch 1/10


KeyboardInterrupt: 

In [ ]:

# Load and preprocess the image
image_path = 'CFD_001.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (112, 112))  # Resize to match your model's input shape
# image = image / 255.0  # Normalize the pixel values
cv2.imwrite( "asg.jpg", image)
# Convert the image to a NumPy array with the expected shape
# input_image = np.expand_dims(image, axis=0)  # Add a batch dimension
prediction = model.predict(input_image)
prediction

NameError: name 'input_image' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

prediction *= 255

prediction_2d = prediction.squeeze()

plt.figure(figsize=(8, 8))
plt.imshow(prediction_2d, cmap='gray')
plt.colorbar()
plt.axis('off')
plt.show()

NameError: name 'prediction' is not defined